In [1]:
%load_ext autoreload
%autoreload 2
    
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import sys

# To view tensorboard metrics
# tensorboard --logdir=logs --port=6006 --bind_all
from torch.utils.tensorboard import SummaryWriter
from functools import partial
from evolver import CrossoverType, MutationType, InitType, MatrixEvolver
from unet import UNet
from dataset_utils import PartitionType
from cuda_utils import maybe_get_cuda_device, clear_cuda
from landcover_dataloader import get_landcover_dataloaders

from ignite.contrib.handlers.tensorboard_logger import *
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, ConfusionMatrix, mIoU
from ignite.handlers import ModelCheckpoint
from ignite.utils import setup_logger
from ignite.engine import Engine

In [2]:
evolver = MatrixEvolver([[4, 4]], CrossoverType.UNIFORM,
                            MutationType.FLIP_BIT, 
                            InitType.BINOMIAL, 
                            flip_bit_prob=0.2, 
                            flip_bit_decay=0.999,
                            binomial_prob=0.8)

template = np.array([
    [1, 0, 0, 1],
    [0, 1, 0, 0],
    [0, 0, 1, 1],
    [1, 0, 0, 1],
])
num_generations = 100
num_children = 200
for gen in range(0, num_generations):
    print(evolver.flip_bit_prob)
    for child_num in range(0, num_children):
        child = evolver.spawn_child()
        loss = 1.0 / np.sum(np.abs(child - template))
        if gen == num_generations - 1:
            print(gen, child_num, loss)
            print(child)
        evolver.add_child(child, loss)
    evolver.update_parents()

0.2
0.1998
0.1996002
0.1994005998
0.1992011992002
0.1990019980009998
0.1988029960029988
0.1986041930069958
0.19840558881398881
0.19820718322517483
0.19800897604194967
0.19781096706590773
0.1976131560988418
0.19741554294274297
0.1972181273998002
0.1970209092724004
0.196823888363128
0.19662706447476488
0.19643043741029012
0.19623400697287982
0.19603777296590694
0.19584173519294104
0.1956458934577481
0.19545024756429036
0.19525479731672607
0.19505954251940935
0.19486448297688994


/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


0.19466961849391304
0.19447494887541913
0.1942804739265437
0.19408619345261716
0.19389210725916453
0.19369821515190536
0.19350451693675344
0.19331101241981669
0.19311770140739687
0.19292458370598947
0.1927316591222835
0.1925389274631612
0.19234638853569805
0.19215404214716236
0.1919618881050152
0.19176992621691016
0.19157815629069325
0.19138657813440255
0.19119519155626816
0.19100399636471188
0.19081299236834717
0.19062217937597883
0.19043155719660285
0.19024112563940623
0.19005088451376684
0.18986083362925307
0.1896709727956238
0.18948130182282819
0.18929182052100535
0.18910252870048436
0.18891342617178386
0.18872451274561208
0.18853578823286646
0.1883472524446336
0.18815890519218897
0.18797074628699678
0.1877827755407098
0.18759499276516908
0.18740739777240392
0.18721999037463152
0.1870327703842569
0.18684573761387263
0.18665889187625875
0.1864722329843825
0.18628576075139813
0.18609947499064675
0.1859133755156561
0.18572746214014044
0.1855417346780003
0.1853561929433223
0.1851708367

In [3]:
# Define directories for data, logging and model saving.
base_dir = os.getcwd()
dataset_name = "landcover_large"
dataset_dir = os.path.join(base_dir, "data/" + dataset_name)

experiment_name = "dropout_finetuning"
model_name = "best_model_30_validation_accuracy=0.9409.pt"
model_path = os.path.join(base_dir, "logs/" + dataset_name + "/" + model_name)
log_dir = os.path.join(base_dir, "logs/" + dataset_name + "_" + experiment_name)

# Create DataLoaders for each partition of Landcover data.
dataloader_params = {
    'batch_size': 16,
    'shuffle': True,
    'num_workers': 6,
    'pin_memory': True}

partition_types = [PartitionType.TRAIN, PartitionType.VALIDATION, 
                   PartitionType.FINETUNING, PartitionType.TEST]
data_loaders = get_landcover_dataloaders(dataset_dir, 
                                         partition_types,
                                         dataloader_params,
                                         force_create_dataset=False)

train_loader = data_loaders[0]
finetuning_loader = data_loaders[2]
test_loader = data_loaders[3]

# Get GPU device if available.
device = maybe_get_cuda_device()

# Determine model and training params.
params = {
    'max_epochs': 10,
    'n_classes': 4,
    'in_channels': 4,
    'depth': 5,
    'learning_rate': 0.01,
    'log_steps': 1,
    'save_top_n_models': 4,
    'num_children': 3
}

clear_cuda()    
model = UNet(in_channels = params['in_channels'],
             n_classes = params['n_classes'],
             depth = params['depth'])
model.load_state_dict(torch.load(model_path))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=params['learning_rate'])

# Determine metrics for evaluation.
train_metrics = {
        "accuracy": Accuracy(), 
        "loss": Loss(criterion),
        "mean_iou": mIoU(ConfusionMatrix(num_classes = params['n_classes'])),
        }

validation_metrics = {
        "accuracy": Accuracy(), 
        "loss": Loss(criterion),
        "mean_iou": mIoU(ConfusionMatrix(num_classes = params['n_classes'])),

}

for batch in train_loader:
    batch_x = batch[0]
    _ = model(batch_x)
    break
    
drop_out_layers = model.get_dropout_layers()
del model, batch_x
clear_cuda()


In [4]:
global_best_child = None
for layer in drop_out_layers:
    if layer.name != "down_4":
        continue

    layer_name = layer.name
    size = layer.x_size[1:]
    sizes = [size]
    clear_cuda()    
    model = UNet(in_channels = params['in_channels'],
                 n_classes = params['n_classes'],
                 depth = params['depth'])
    model.load_state_dict(torch.load(model_path))

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=params['learning_rate'])

    evolver = MatrixEvolver(sizes, 
                            CrossoverType.UNIFORM,
                            MutationType.FLIP_BIT, 
                            InitType.BINOMIAL, 
                            flip_bit_prob=0.5, 
                            flip_bit_decay=0.99,
                            binomial_prob=0.75)


    log_dir_test = log_dir + "_evolver_testing_" + layer_name

    def dropout_finetune_step(engine, batch):
        with torch.no_grad():
            batch_x, batch_y = batch
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            loss = sys.float_info.max
            for i in range(params['num_children']):
                child_mask = evolver.spawn_child()
                model.set_dropout_masks({layer_name: torch.tensor(child_mask[0], dtype=torch.float32).to(device)})
                outputs = model(batch_x)
                current_loss = criterion(outputs, batch_y).item()
                # Priorities stored in a min heap by evolver and so
                # we use the reciprocal.
                evolver.add_child(child_mask, 1.0 / current_loss)
                loss = min(loss, current_loss)
            
            best_child = evolver.get_best_child()
            global global_best_child
            global_best_child = best_child
            print("update", np.count_nonzero(global_best_child[0]))
            model.set_dropout_masks({layer_name: torch.tensor(best_child[0], dtype=torch.float32).to(device)})
            return loss


    # Create Trainer or Evaluators
    trainer = Engine(dropout_finetune_step)
    train_evaluator = create_supervised_evaluator(model, metrics=train_metrics, device=device)


    validation_evaluator = create_supervised_evaluator(model, metrics=validation_metrics, device=device)

    trainer.logger = setup_logger("Trainer")
    train_evaluator.logger = setup_logger("Train Evaluator")


    validation_evaluator.logger = setup_logger("Validation Evaluator")


    @trainer.on(Events.ITERATION_COMPLETED(every=1))
    def report_evolver_stats(engine):
        priorities = np.array(evolver.get_generation_priorities())
        # Take reciprocal since we needed to store priorities in min heap.
        priorities = 1.0 / priorities
        tb_logger.writer.add_scalar("training/evolver_count",
                                    priorities.shape[0], engine.state.iteration)
        tb_logger.writer.add_scalar("training/evolver_mean",
                                    np.mean(priorities), engine.state.iteration)
        tb_logger.writer.add_scalar("training/evolver_std",
                                    np.std(priorities), engine.state.iteration)
        evolver.update_parents()
        
    # Tensorboard Logger setup below based on pytorch ignite example
    # https://github.com/pytorch/ignite/blob/master/examples/contrib/mnist/mnist_with_tensorboard_logger.py
    @trainer.on(Events.EPOCH_COMPLETED)
    def compute_metrics(engine):
        """Callback to compute metrics on the train and validation data."""
        global global_best_child
        for l in model.get_dropout_layers():
            if l.name == "down_4":
                print("PRE")
                print("non zero", np.count_nonzero(global_best_child[0]))
                print("eq", torch.sum(torch.eq(torch.tensor(global_best_child[0], dtype=torch.float32).to(device), l.mask)))
                print("size", np.product(size))
        train_evaluator.run(finetuning_loader)
        validation_evaluator.run(test_loader)
        for l in model.get_dropout_layers():
            if l.name == "down_4":
                print("POST")
                print("non zero", np.count_nonzero(global_best_child[0]))
                print("eq",torch.sum(torch.eq(torch.tensor(global_best_child[0], dtype=torch.float32).to(device), l.mask)))
                print("size", np.product(size))

    def score_function(engine):
        """Function to determine the metric upon which to compare model."""
        return engine.state.metrics["accuracy"]

    # Setup Tensor Board Logging    
    tb_logger = TensorboardLogger(log_dir=log_dir_test)

    tb_logger.attach_output_handler(
        trainer,
        event_name=Events.ITERATION_COMPLETED(every=params['log_steps']),
        tag="training",
        output_transform=lambda loss: {"batchloss": loss},
        metric_names="all",
    )

    for tag, evaluator in [("training", train_evaluator), ("validation", validation_evaluator)]:
        tb_logger.attach_output_handler(
            evaluator,
            event_name=Events.EPOCH_COMPLETED,
            tag=tag,
            metric_names="all",
            global_step_transform=global_step_from_engine(trainer),
        )

    tb_logger.attach_opt_params_handler(trainer, 
                                        event_name=Events.ITERATION_COMPLETED(every=params['log_steps']), 
                                        optimizer=optimizer)

    model_checkpoint = ModelCheckpoint(
        log_dir_test,
        n_saved=params['save_top_n_models'],
        filename_prefix="best",
        score_function=score_function,
        score_name="validation_accuracy",
        global_step_transform=global_step_from_engine(trainer),
    )

    validation_evaluator.add_event_handler(Events.COMPLETED, model_checkpoint, {"model": model})
    trainer.run(finetuning_loader, max_epochs=params['max_epochs'])
    tb_logger.close()

2020-08-17 17:29:35,397 Trainer INFO: Engine run starting with max_epochs=10.


update 65816
update 65861
update 65816
update 65480
update 65282
update 65263
update 65320
update 65399
update 65807
update 65396
update 65743
update 65262
update 65471
update 65477
update 65444
update 65569
update 65361
update 65620
update 65736
update 65240
update 65746
update 65670
update 65522
update 65457
update 65822
update 65452
update 65398
update 65301
update 65261
update 65240
update 65386


2020-08-17 17:30:30,624 Train Evaluator INFO: Engine run starting with max_epochs=1.


update 65584
PRE
non zero 65584
eq tensor(131072, device='cuda:0')
size 131072


2020-08-17 17:30:54,285 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:23
2020-08-17 17:30:54,288 Train Evaluator INFO: Engine run complete. Time taken: 00:00:24
2020-08-17 17:30:54,325 Validation Evaluator INFO: Engine run starting with max_epochs=1.
2020-08-17 17:32:51,739 Validation Evaluator INFO: Epoch[1] Complete. Time taken: 00:01:57
2020-08-17 17:32:52,286 Validation Evaluator INFO: Engine run complete. Time taken: 00:01:57
2020-08-17 17:32:52,321 Trainer INFO: Epoch[1] Complete. Time taken: 00:00:55


...

POST
non zero 65506
eq tensor(131072, device='cuda:0')
size 131072
update 65558
update 65541
update 65713
update 65729
update 65727
update 65460
update 65492
update 65558
update 65515
update 65498
update 65584
update 65548
update 65611
update 65579
update 65633
update 65691
update 65663
update 65613
update 65651
update 65642
update 65465
update 65396
update 65535
update 65573
update 65542
update 65354
update 65448
update 65493
update 65472
update 65614
update 65752


2020-08-17 17:59:58,344 Train Evaluator INFO: Engine run starting with max_epochs=1.


update 65842
PRE
non zero 65842
eq tensor(131072, device='cuda:0')
size 131072


2020-08-17 18:00:22,242 Train Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:24
2020-08-17 18:00:22,245 Train Evaluator INFO: Engine run complete. Time taken: 00:00:24
2020-08-17 18:00:22,278 Validation Evaluator INFO: Engine run starting with max_epochs=1.
2020-08-17 18:02:19,973 Validation Evaluator INFO: Epoch[1] Complete. Time taken: 00:01:58
2020-08-17 18:02:19,976 Validation Evaluator INFO: Engine run complete. Time taken: 00:01:58
2020-08-17 18:02:20,011 Trainer INFO: Epoch[10] Complete. Time taken: 00:00:54
2020-08-17 18:02:20,012 Trainer INFO: Engine run complete. Time taken: 00:32:45


POST
non zero 65842
eq tensor(131072, device='cuda:0')
size 131072


In [5]:
global_best_child

[array([[[0., 1., 1., ..., 1., 1., 0.],
         [1., 0., 0., ..., 1., 0., 0.],
         [1., 1., 0., ..., 1., 1., 0.],
         ...,
         [1., 1., 0., ..., 0., 0., 1.],
         [1., 1., 0., ..., 1., 0., 0.],
         [1., 0., 0., ..., 0., 0., 1.]],
 
        [[1., 0., 1., ..., 0., 0., 0.],
         [1., 0., 1., ..., 1., 0., 1.],
         [1., 1., 1., ..., 0., 0., 0.],
         ...,
         [0., 1., 1., ..., 0., 0., 0.],
         [0., 1., 1., ..., 1., 0., 0.],
         [1., 1., 0., ..., 1., 1., 1.]],
 
        [[1., 0., 0., ..., 1., 1., 1.],
         [0., 1., 0., ..., 1., 0., 0.],
         [0., 0., 1., ..., 0., 0., 1.],
         ...,
         [1., 1., 1., ..., 0., 0., 1.],
         [1., 1., 1., ..., 0., 1., 1.],
         [1., 1., 0., ..., 1., 1., 0.]],
 
        ...,
 
        [[1., 1., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 1., 0., 0.],
         [0., 1., 1., ..., 1., 0., 1.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [1., 1., 0., ..., 1., 0., 0.],
